# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [6]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [7]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [8]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [18]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,
                                             verbose=False)

Find what features had non-zero weight.

In [11]:
model_all.get('coefficients')

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


## Q3: for all features, when L1 penalty = 1e10,
## these features are zero: bedrooms, bedrooms_square, sqft_lot, sqft_lot_sqrt, floors, floors_square
## these features are non-zero: intercept(274873), bathrooms(8468.53), sqft_living(24.42), sqft_living_sqrt(350.06)

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [15]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [16]:
import numpy as np

In [17]:
l1_range = np.logspace(1,7,num=13)
l1_range

array([  1.00000000e+01,   3.16227766e+01,   1.00000000e+02,
         3.16227766e+02,   1.00000000e+03,   3.16227766e+03,
         1.00000000e+04,   3.16227766e+04,   1.00000000e+05,
         3.16227766e+05,   1.00000000e+06,   3.16227766e+06,
         1.00000000e+07])

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [27]:
model_tmp = graphlab.linear_regression.create(training, features=all_features,
                                             target='price',
                                             l1_penalty=l1_range[5],
                                             l2_penalty=0,
                                             validation_set=None, verbose=False)
rmse = model_tmp.evaluate(validation)['rmse']
rss = ((model_tmp.evaluate(validation)['rmse']) ** 2) * len(validation)
print rmse, rss, len(validation)

254847.414528 6.25766317206e+14 9635


In [41]:
rss_L1 = {}
rss_L1_fr = graphlab.SFrame()
for l1_penalty_val in l1_range:
    model_1 = graphlab.linear_regression.create(training, features=all_features,
                                                target='price',
                                                l1_penalty=l1_penalty_val,
                                                l2_penalty=0,
                                                validation_set=None, verbose=False)
    # evaluate the model on validation set
    rss_L1[l1_penalty_val] = ((model_1.evaluate(validation)['rmse']) ** 2) * len(validation)
rss_L1_fr['L1_penalty'] = rss_L1.keys()
rss_L1_fr['RSS'] = rss_L1.values()
rss_chosen = rss_L1_fr[rss_L1_fr['RSS']==min(rss_L1_fr['RSS'])]
print rss_L1
print rss_L1_fr
print rss_chosen

{100000.0: 625767302791634.9, 316227.76601683791: 625769507643886.5, 100.0: 625766286057885.4, 3162.2776601683795: 625766317206080.5, 31.622776601683793: 625766285362393.8, 1000.0: 625766295212186.8, 10.0: 625766285142461.0, 31622.776601683792: 625766606749278.9, 10000.0: 625766386760657.8, 3162277.6601683795: 625799062845467.1, 316.22776601683796: 625766288257225.0, 1000000.0: 625776517727024.8, 10000000.0: 625883719085424.9}
+---------------+-------------------+
|   L1_penalty  |        RSS        |
+---------------+-------------------+
|    100000.0   | 6.25767302792e+14 |
| 316227.766017 | 6.25769507644e+14 |
|     100.0     | 6.25766286058e+14 |
| 3162.27766017 | 6.25766317206e+14 |
| 31.6227766017 | 6.25766285362e+14 |
|     1000.0    | 6.25766295212e+14 |
|      10.0     | 6.25766285142e+14 |
| 31622.7766017 | 6.25766606749e+14 |
|    10000.0    | 6.25766386761e+14 |
| 3162277.66017 | 6.25799062845e+14 |
+---------------+-------------------+
[13 rows x 2 columns]
Note: Only the 

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

## Q6: when l1_penalty = 10.0, rss is the min = 6.257662851e+14

In [47]:
# test the l1_penalty on test data
model_2 = graphlab.linear_regression.create(training, features=all_features,
                                           target='price',
                                           l1_penalty=rss_chosen['L1_penalty'],
                                           l2_penalty=0,
                                           validation_set=None, verbose=False)
rss_test = ((model_2.evaluate(testing)['rmse'])**2) * len(testing)
print "%.4g" %rss_test 

9.852e+13


In [50]:
# count how many non-zero coefficients
coeff = model_2.get('coefficients')
coeff.print_rows(num_rows=len(coeff))
np.count_nonzero(coeff['value'])

+------------------+-------+----------------+---------------+
|       name       | index |     value      |     stderr    |
+------------------+-------+----------------+---------------+
|   (intercept)    |  None | 6870610.30709  | 200759.223045 |
|     bedrooms     |  None | -15608.7364849 | 4306.69852875 |
| bedrooms_square  |  None | 490.449090676  | 261.781319384 |
|    bathrooms     |  None | 55524.1822706  | 4973.32493695 |
|   sqft_living    |  None | -662.501053583 |      nan      |
| sqft_living_sqrt |  None | -52723.4887044 | 1566.60027457 |
|     sqft_lot     |  None | 1.04698310583  | 0.12739915556 |
|  sqft_lot_sqrt   |  None | -869.393478399 | 79.8993132762 |
|      floors      |  None | -74062.175802  | 26717.6634879 |
|  floors_square   |  None | 25866.5409727  | 7675.68986791 |
|    waterfront    |  None | 630120.194551  | 25328.7927187 |
|       view       |  None | 36533.5599793  | 3153.46454277 |
|    condition     |  None | 27790.5045829  | 3591.53140949 |
|      g

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?